## **5. LDA Models**

#### **Step 1: Persiapkan Kamus**

Pada langkah ini, dilakukan pembentukan kamus (dictionary) berdasarkan data lirik lagu yang akan dianalisis. Kamus ini akan digunakan untuk mengidentifikasi kata-kata unik dalam dataset. Selain itu, juga dilakukan proses filtering berdasarkan frekuensi kemunculan kata. Kata-kata yang jarang muncul atau terlalu umum (seperti kata penghubung) akan dihilangkan dari kamus. Tujuan dari langkah ini adalah untuk mempersiapkan data yang akan digunakan dalam pembentukan model LDA (Latent Dirichlet Allocation).

Pada prosesnya, kami menggunakan token kata yang sudah di pickle sebelumnya bernama `tokenized.pkl` dan menggunakan pustaka **gensim** sebagai representasi kamus dari dokumen.

In [35]:
import pickle
from gensim.corpora import Dictionary

with open('pickle/tokenized.pkl', 'rb') as f:
    texts = pickle.load(f)

dictionary = Dictionary(texts)
print(dictionary)

Dictionary<2538 unique tokens: ['adore', 'armor', 'arrow', 'atone', 'black']...>


#### **Step 2: Bag-of-Words**

Setelah kamus terbentuk, langkah selanjutnya adalah mengubah teks lirik lagu menjadi representasi Bag-of-Words (BoW). Representasi ini menghitung frekuensi kemunculan setiap kata dalam setiap dokumen. Hasilnya adalah matriks sparse yang merepresentasikan dataset dalam bentuk vektor kata. Selain itu, juga dilakukan konversi indeks kata dalam kamus menjadi kata yang sesuai. Hal ini berguna untuk mempermudah interpretasi hasil dari model LDA (Latent Dirichlet Allocation).

In [42]:
gensim_corpus = [dictionary.doc2bow(song) for song in texts]
temp = dictionary[0]
id2word = dictionary.id2token

#### **Step 8: Parameter Models**

Langkah ini melibatkan penentuan parameter yang akan digunakan dalam model LDA. Parameter yang harus diatur meliputi jumlah topik yang diinginkan, jumlah iterasi, dan metode penyesuaian (adjustment) lainnya. Pemilihan parameter yang tepat sangat penting untuk menghasilkan model LDA yang akurat dan relevan dengan dataset yang dianalisis.

Pustaka **Gensim** memungkinkan kami mengatur beberapa parameter pemodelan, seperti:
1. `num_topics` = terkait berapa banyak topik ingin ditemukan. Kami sepakat untuk mengidentifikasi 5 topik, 
2. `chunksize` = jumlah dokumen yang dipertimbangkan dalam setiap siklus pelatihannya,
3. `passes` = jumlah melewati korpus literatur selama pelatihan, 
4. `iterations` = jumlah maksimum iterasi

In [43]:
chunksize = 1000
passes = 20
iterations = 500
num_topics = 5

#### **Step 9: Training Model**

Setelah menentukan parameter, langkah selanjutnya adalah melatih model LDA menggunakan representasi BoW dari dataset lirik lagu. Proses pelatihan ini akan memetakan setiap dokumen ke dalam distribusi topik yang relevan. Model LDA akan mengidentifikasi topik-topik yang muncul secara implisit dalam dataset berdasarkan distribusi kata-kata yang diamati.

In [44]:
from gensim.models import LdaModel

lda_model = LdaModel(
    corpus=gensim_corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes
)

Setelah model LDA dilatih, kami menyimpan model yang sudah jadi kedalam pickle, hal ini kami lakukan karena skenario Evaluasi model berada pada file notebook yang terpisah, sehingga model ini bila langsung digunakan pada project yang berbeda.

In [45]:
# with open('pickle/lda_model.pkl', 'wb') as f:
#     pickle.dump(lda_model, f)

#### **Step 10: Visualisasi Model LDA**

Setelah model LDA terlatih, langkah terakhir adalah melakukan visualisasi hasil model menggunakan library `pyLDAvis`. Visualisasi ini membantu kami dalam memahami dan mengeksplorasi topik-topik yang dihasilkan oleh model. PyLDAvis menyediakan tampilan interaktif yang menampilkan distribusi topik, kata-kata yang paling berkontribusi pada setiap topik, serta tampilan hubungan antara topik-topik. Dengan visualisasi ini, kita dapat menggali wawasan baru tentang pola tematik dalam lirik lagu dan memperoleh pemahaman yang lebih baik tentang konten dan makna di balik dataset tersebut.

In [46]:
# %pip install pyLDAvis

In [47]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

vis_data = gensimvis.prepare(lda_model, gensim_corpus, dictionary)
pyLDAvis.display(vis_data)

Dari hasil percobaan pertama ini, kami menyimpulkan bahwa pembagian topik dari token-token yang dihasilkan setiap topiknya adalah sebagai berikut :

**(1) Topik 1 - Kehilangan / Perpisahan**
- Kata Kunci: **away, far, back, gone, time, day, tonight, never, alone, one, left, heart, mind**

**(2) Topik 2 - Kehidupan**
- Kata Kunci: **time, life, nothing, end, lost, find, day, heart, mind, nightmare**

**(3) Topik 3 - Religion**
- Kata Kunci: **god, life, without, dead, die, light, need**

**(4) Topik 4 - Perang dan Konflik**
- Kata Kunci: **war, take, world, voice, fear, lost, life, die, shadow**

**(5) Topik 5 - Kegilaan dan Ketidakwarasan**
- Kata Kunci: **insane, almost, alone, tell, creating**

Perlu diingat bahwa ini adalah interpretasi subjektif berdasarkan kata kunci yang diberikan. Penentuan topik dapat bervariasi tergantung pada konteks dan interpretasi individu.